ND

In [6]:
import numpy as np
import copy
import os
import yaml
yaml.Dumper.ignore_aliases = lambda *args : True
import utils

import importlib
importlib.reload(utils)

<module 'utils' from '/n/home12/cfpark00/ML/ToyCompDiff/utils.py'>

In [7]:
config_nd={
    'experiment_directory': "/n/holylfs06/LABS/finkbeiner_lab/Users/cfpark00/datadir/ML/vectorCG/info_scale/info_scale_1",
    "dataset": "vec",
    'seed': None,
    "dim": 3,
    'n_classes': 8,
    'means': [
        [-1.,-1.,-1],
        [-1.,-1.,1.],
        [-1.,1.,-1.],
        [-1.,1.,1.],
        [1.,-1.,-1],
        [1.,-1.,1],
        [1.,1.,-1],
        [1.,1.,1]
        ],
    'covs': [
        [[0.03,0.0,0.0],
         [0.0,0.03,0.0],
         [0.0,0.0,0.03]],
    ]*8,
    'noise_covs':[
        [[0.0,0.0,0.0],
         [0.0,0.0,0.0],
         [0.0,0.0,0.0]], 
    ]*8,
    'n_samples_train':[4096,4096,4096,0,4096,0,0,0],
    'n_samples_train_gen':[4096,4096,4096,0,4096,0,0,0],
    'n_samples_test':[0,0,0,4096,0,4096,4096,4096],
    'n_samples_test_gen':[0,0,0,4096,0,4096,4096,4096],
    "model_params":{
        "model_type": "VDiff",
        "network_type": "MLP",
        "hidden_dims": [1024],
        "init_scale": 1.0,
        "optimizer_type": "AdamW",
        "optimizer_params": {"lr": 0.001,"weight_decay": 0.01},
        "data_noise":1e-3,
        "beta_settings": {"type":"logsnr","noise_schedule":"learned_linear","gamma_min":-5, "gamma_max":10},
    },
    "batch_size": 128,
    "train_ratio": 0.85,
    'num_steps': 20000,
    'save_steps': 15,

    #figure parameters
    "fig_x": [[1.,0.,0.],[1.,0.,0.]],
    "fig_y": [[0.,1.,0.],[0.,0.,1.]],
}

In [8]:
def get_config(dim_params):
    dim=dim_params["dim"]
    dim_nuisance=dim_params["dim_nuisance"]
    n_per_train=dim_params["n_per_train"]
    n_per_gen=dim_params["n_per_gen"]
    perdim=dim_params["perdim"]
    if isinstance(perdim,list):
        perdim=np.array(perdim)
    else:
        perdim=np.full(dim,perdim)
    m=dim_params["m"]
    if isinstance(m,list):
        m=np.array(m)
    else:
        m=np.full(dim,m)
    scale=(perdim-1)*m
    scalesq=scale[None,:]*scale[:,None]
    s=dim_params["s"]
    if isinstance(s,list):
        s=np.array(s)
    else:
        s=np.full(dim,s)
    s_n=dim_params["s_n"]
    if isinstance(s,list):
        s_n=np.array(s_n)
    else:
        s_n=np.full(dim,s_n)
    batch_size=dim_params["batch_size"]
    assert n_per_train%batch_size==0
    assert n_per_gen%batch_size==0
    

    ######
    n_classes=int(np.prod(perdim))
    inds=np.stack(np.meshgrid(*[np.arange(d) for d in perdim],indexing="ij"),axis=-1)
    means=inds.astype(np.float32)*m
    if "translate" in dim_params:
        means+=np.array(dim_params["translate"])
    means/=scale
    means=means.reshape(-1,dim).tolist()
    cov=(np.eye(dim)*s[:,None]*s[None,:])/scalesq
    cov=cov.tolist()
    covs=[cov]*n_classes
    noise_cov=(np.eye(dim)*s_n[:,None]*s_n[None,:])/scalesq
    noise_cov=noise_cov.tolist()
    noise_covs=[noise_cov]*n_classes
    ##
    if "train_inds" not in dim_params:
        n_nz=(inds!=0).sum(-1)
        train_inds=np.logical_or(n_nz==0,n_nz==1).flatten()
    else:
        train_inds=dim_params["train_inds"]
        if train_inds.dtype!=bool:
            train_inds_=train_inds
            train_inds=np.zeros(n_classes,dtype=bool)
            train_inds[train_inds_]=True
    test_inds=np.logical_not(train_inds)
    n_samples_train=np.zeros(n_classes,dtype=np.int32)
    n_samples_train[train_inds]=n_per_train
    n_samples_train=n_samples_train.tolist()
    n_samples_test=np.zeros(n_classes,dtype=np.int32)
    n_samples_test[~train_inds]=n_per_gen#intentional
    n_samples_test=n_samples_test.tolist()

    n_samples_train_gen=np.zeros(n_classes,dtype=np.int32)
    n_samples_train_gen[train_inds]=n_per_gen
    n_samples_train_gen=n_samples_train_gen.tolist()
    n_samples_test_gen=np.zeros(n_classes,dtype=np.int32)
    n_samples_test_gen[~train_inds]=n_per_gen
    n_samples_test_gen=n_samples_test_gen.tolist()
    #
    fig_x=np.zeros((dim-1,dim+dim_nuisance))
    fig_y=np.zeros((dim-1,dim+dim_nuisance))
    for j_ax in range(dim-1):
        fig_x[j_ax,0]=1
        fig_y[j_ax,j_ax+1]=1
    fig_x=fig_x.tolist()
    fig_y=fig_y.tolist()

    config_=copy.deepcopy(config_nd)

    #global
    config_["perdim"]=perdim.tolist()
    config_["dim"]=dim
    config_["dim_nuisance"]=dim_nuisance
    

    #data
    config_["inds_md"]=inds.reshape(-1,dim).tolist()
    config_["means"]=means
    config_["covs"]=covs
    config_["noise_covs"]=noise_covs
    config_["n_samples_train"]=n_samples_train
    config_["n_samples_train_gen"]=n_samples_train_gen
    config_["n_samples_test"]=n_samples_test
    config_["n_samples_test_gen"]=n_samples_test_gen
    config_["fig_x"]=fig_x
    config_["fig_y"]=fig_y

    #model
    config_["n_classes"]=n_classes
    config_["batch_size"]=batch_size

    return config_


In [9]:
dim_params=dict(dim=2,
dim_nuisance=0,
perdim=2,
m=1.0,
s=0.05,
s_n=0.005,
n_per_train=1024,
n_per_gen=128,
batch_size=32
)

In [49]:
perdim=4
dim=2
for seed in range(5):
    experiment_name=f"seed={seed}"
    fol=f"base"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=600_000
    config_['save_steps']=60
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [7]:
perdim=4
dim=2
for seed in range(5):
    experiment_name=f"seed={seed}"
    fol=f"base_hwd"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=600_000
    config_['save_steps']=60
    config_["model_params"]["optimizer_params"]["weight_decay"]=0.05
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [8]:
perdim=4
dim=2
for seed in range(5):
    experiment_name=f"seed={seed}"
    fol=f"base_hhwd"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=600_000
    config_['save_steps']=60
    config_["model_params"]["optimizer_params"]["weight_decay"]=0.5
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [10]:
perdim=4
dim=2
for seed in range(5):
    experiment_name=f"seed={seed}"
    fol=f"hard"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["s"]=0.01
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=600_000
    config_['save_steps']=60
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [11]:
perdim=4
dim=2
for seed in range(5):
    experiment_name=f"seed={seed}"
    fol=f"hard_hwd"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["s"]=0.01
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=600_000
    config_['save_steps']=60
    config_["model_params"]["optimizer_params"]["weight_decay"]=0.05
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [50]:
perdim=4
dim=2
for seed in range(3):
    experiment_name=f"seed={seed}"
    fol=f"elong"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=1_500_000
    config_['save_steps']=80
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [5]:
perdim=4
dim=2
for seed in range(3):
    experiment_name=f"seed={seed}"
    fol=f"elong_nowd"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=1_500_000
    config_['save_steps']=80
    config_["model_params"]["optimizer_params"]["weight_decay"]=0.0
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [12]:
perdim=4
dim=2
for seed in range(3):
    experiment_name=f"seed={seed}"
    fol=f"elong_wd0.5"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_['num_steps']=1_500_000
    config_['save_steps']=80
    config_["model_params"]["optimizer_params"]["weight_decay"]=0.005
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [27]:
perdim=4
dim=2
for seed in range(5):
    experiment_name=f"seed={seed}"
    fol=f"det"

    ###########settings
    dim_params_=copy.deepcopy(dim_params)
    dim_params_["perdim"]=perdim
    dim_params_["dim"]=dim
    
    config_=get_config(dim_params_)
    config_["seed"]=seed
    config_["model_params"]["model_type"]="Det"
    config_['num_steps']=600_000
    config_['save_steps']=60
    ##############
    
    config_=utils.process_config(config_,check_only=True)
    experiment_directory=os.path.join("data",fol,experiment_name)
    config_["experiment_directory"]=experiment_directory
    yaml_fol=os.path.join("yamls",fol)
    os.makedirs(yaml_fol,exist_ok=True)
    yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
    yaml.dump(config_,open(yaml_path,"w"))

In [25]:
perdim=4
dim=2
for dn in [1,3,5]:
    for seed in range(3):
        experiment_name=f"dn={dn}_seed={seed}"
        fol=f"nuisance"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["dim_nuisance"]=dn
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim

        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [42]:
perdim=4
dim=2
for dn in [1,3,5]:
    for seed in range(3):
        experiment_name=f"dn={dn}_seed={seed}"
        fol=f"nlnuisance"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["dim_nuisance"]=dn
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim

        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        config_["nl_nuisance"]=True
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [43]:
perdim=4
dim=2
for dn in [1,3,5]:
    for seed in range(3):
        experiment_name=f"dn={dn}_seed={seed}"
        fol=f"nuisance_det"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["dim_nuisance"]=dn
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim

        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_["model_params"]["model_type"]="Det"
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [44]:
perdim=4
dim=2
for dn in [1,3,5]:
    for seed in range(3):
        experiment_name=f"dn={dn}_seed={seed}"
        fol=f"nlnuisance_det"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["dim_nuisance"]=dn
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim

        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_["model_params"]["model_type"]="Det"
        config_['num_steps']=600_000
        config_['save_steps']=60
        config_["nl_nuisance"]=True
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [ ]:
dim=2
for perdim in [3,5,6]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim={perdim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [5]:
perdim=4
for dim in [3,4,5]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=4_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [8]:
perdim=2
for dim in [2,3,4,5]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=2_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [10]:
perdim=2
for dim in [3]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=2_dim={dim}_inis=0.3_twd"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        config_["model_params"]["init_scale"]=0.3
        config_["model_params"]["optimizer_params"]["weight_decay"]=0.02
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [9]:
perdim=[2,4]
for dim in [2]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=24_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [10]:
perdim=4
for dim in [2]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=4_aniso_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["s"]=[0.05,0.5]
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [11]:
perdim=4
for dim in [2]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=4_aniso2_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["s"]=[0.025,0.1]
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [13]:
perdim=4
for dim in [2]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=4_aniso3_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["s"]=[0.05,0.2]
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [14]:
perdim=4
for dim in [2]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=4_aniso4_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["s"]=[0.01,0.15]
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))

In [5]:
perdim=4
for dim in [3]:
    for seed in range(3):
        experiment_name=f"seed={seed}"
        fol=f"perdim=4_aniso4_dim={dim}"

        ###########settings
        dim_params_=copy.deepcopy(dim_params)
        dim_params_["perdim"]=perdim
        dim_params_["s"]=[0.01,0.01,0.15]
        dim_params_["dim"]=dim
        
        config_=get_config(dim_params_)
        config_["seed"]=seed
        config_['num_steps']=600_000
        config_['save_steps']=60
        ##############
        
        config_=utils.process_config(config_,check_only=True)
        experiment_directory=os.path.join("data",fol,experiment_name)
        config_["experiment_directory"]=experiment_directory
        yaml_fol=os.path.join("yamls",fol)
        os.makedirs(yaml_fol,exist_ok=True)
        yaml_path=os.path.join(yaml_fol,experiment_name+".yaml")
        yaml.dump(config_,open(yaml_path,"w"))